In [249]:
import numpy as np
import argparse
import cv2
import os

In [286]:
original =  cv2.imread('C:\\Users\Cheon\MasterData\mLForward\sfn52.png'
                 );

In [287]:
cv2.imshow('color image',original) 
cv2.waitKey(0); cv2.destroyAllWindows(); cv2.waitKey(1)

-1

In [288]:
width = 512
height = 512
dim = (width, height)
 
# resize image
resized = cv2.resize(original, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 
cv2.imshow("Resized image", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Resized Dimensions :  (512, 512, 3)


In [305]:
# define the upper and lower boundaries of the HSV pixel
# intensities to be considered 'skin'

#sa values
#lower = np.array([60, 50, 20], dtype = "uint8")
#upper = np.array([200, 255, 255], dtype = "uint8")

#sb values
#lower = np.array([90, 30, 20], dtype = "uint8")
#upper = np.array([180, 255, 255], dtype = "uint8")

#sc values
#lower = np.array([30, 5,0], dtype = "uint8")
#upper = np.array([180, 255, 160], dtype = "uint8")

#sd values
#lower = np.array([20, 20, 100], dtype = "uint8")
#upper = np.array([180, 255, 200], dtype = "uint8")

#se values
#lower = np.array([90, 70, 70], dtype = "uint8")
#upper = np.array([180, 255, 255], dtype = "uint8")

#sf values
lower = np.array([60, 60, 60], dtype = "uint8")
upper = np.array([180, 255, 255], dtype = "uint8")

In [306]:
converted = cv2.cvtColor(resized, cv2.COLOR_RGB2HSV)
blur = cv2.blur(converted,(5,5))
skinMask = cv2.inRange(converted, lower, upper)
cv2.imshow('result', skinMask)
cv2.waitKey()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [307]:
# apply a series of erosions and dilations to the mask
# using an elliptical kernel
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
skinMask = cv2.erode(skinMask, kernel, iterations = 2)
skinMask = cv2.dilate(skinMask, kernel, iterations = 5)
skin = cv2.bitwise_and(resized, resized, mask = skinMask)
# show the skin in the image along with the mask
cv2.imshow("images", skinMask)

# show the skin in the image along with the mask
#cv2.imshow("Mask image", skinMask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [274]:
#Test (NOT WORKING) for further shadow removal by taking only larges contour
output = skin
contours, hierarchy = cv2.findContours(skinMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
blank_mask = np.zeros(output.shape, dtype=np.uint8)

if len(contours) != 0:
    # draw in blue the contours that were founded

    # find the biggest countour (c) by the area
    c = max(contours, key = cv2.contourArea)
    
    cv2.fillPoly(blank_mask, [c], (255,255,255))


# show the images
cv2.imshow("Result", blank_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [308]:
#Function to convert images to mask 
#Manually set lower and upper boundaries depending on the userclass
#No one size fit all method

import numpy as np
import argparse
import cv2


def massconvert(imageDir,filename,saveDir):
    original =  cv2.imread(imageDir);
    width = 512
    height = 512
    dim = (width, height)

    # resize image
    resized = cv2.resize(original, dim, interpolation = cv2.INTER_AREA)


    # define the upper and lower boundaries of the HSV pixel
    # intensities to be considered 'skin'
    lower = np.array([60, 60, 60], dtype = "uint8")
    upper = np.array([180, 255, 255], dtype = "uint8")

    converted = cv2.cvtColor(resized, cv2.COLOR_RGB2HSV)
    skinMask = cv2.inRange(converted, lower, upper)

    # apply a series of erosions and dilations to the mask
    # using an elliptical kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skinMask = cv2.erode(skinMask, kernel, iterations = 3)
    skinMask = cv2.dilate(skinMask, kernel, iterations = 5)
    cv2.imwrite(saveDir + '\\PR' + filename, skinMask)
    #print(saveDir + '\\PR' + filename)




In [284]:
#Function to loop through the subdirectory andd convert all images matching prefix(which indicatesa a certain user's feet type)
def conversionLoop(mainDir,subdirectory,prefix):
    import os, re
    pfName = mainDir + subdirectory
    for i in os.listdir(pfName):
        #rint(os.path.join(pfName,i))
        if os.path.isfile(os.path.join(pfName,i)) and i.startswith(prefix):
            #print(pfName)
            massconvert(os.path.join(pfName,i),i,pfName)
            #print(os.path.join(pfName,i))

In [309]:
#Execute conversion loop in all subdirectories
dirList = ['mClose','mLForward','mLTurn','mLUp','mNeutral','mPointIn','mPointOut','mRForward','mRTurn','mRUp']
for i in dirList:
    conversionLoop('C:\\Users\Cheon\MasterData\\' ,i,'sf')

In [6]:
for root, subdirectories, files in os.walk(directory):
    for subdirectory in subdirectories:
        print(subdirectory)

NameError: name 'directory' is not defined

In [ ]:
#Test for auto skin thresholding using foot-at-center as example
